In [ ]:
import pandas as pd
import sqlite3
import requests, io

# Connect to the SQLite database containing health and access data
conn = sqlite3.connect('challenge.db')

# Load relevant health outcome variables from the Food Environment Atlas ('access' table).
# For example, select adult obesity, diabetes, and physical inactivity rates by county.
health_access_df = pd.read_sql_query("""
    SELECT
        FIPS,
        PCT_LACCESS_POP15,
        PCT_LACCESS_LOWI15,
        PCT_LACCESS_SNAP15,
        PCT_LACCESS_HHNV15
    FROM access
""", conn)

health_access_df.head()


,FIPS,PCT_LACCESS_POP15,PCT_LACCESS_LOWI15,PCT_LACCESS_SNAP15,PCT_LACCESS_HHNV15
0,1001,32.062255,11.991125,4.608749,3.351332
1,1003,16.767489,5.424427,1.298900,1.905114
2,1005,22.105560,10.739667,4.303147,4.329378
3,1007,4.230324,2.601627,0.676710,2.821427
4,1009,6.497380,2.880150,0.812727,3.336414
